<a href="https://colab.research.google.com/github/darksister1331/ActionBoard-AI/blob/main/AssemblyAI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pydub
!apt-get install ffmpeg

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
ffmpeg is already the newest version (7:4.4.2-0ubuntu0.22.04.1).
0 upgraded, 0 newly installed, 0 to remove and 35 not upgraded.


### Convert .MA4 file into .wav file

In [ ]:
from pydub import AudioSegment

# Load the .m4a file
audio = AudioSegment.from_file("enter file path", format="m4a")

# Export to .wav format
audio.export("enter file path", format="wav")


<_io.BufferedRandom name='/content/audio.wav'>

### .wav file import

In [ ]:
import os
print(os.path.getsize("enter file path "))  # Should be > 0 bytes


142989356


### Upload audio file to AssemblyAI

In [ ]:
import requests

api_key = "b9d094c60c6a4aa78fd4d54feadc7d73"  # Replace with your API key
headers = {'authorization': api_key}

def upload_audio(file_path):
    with open(file_path, 'rb') as f:
        response = requests.post("https://api.assemblyai.com/v2/upload", headers=headers, data=f)
    return response.json()["upload_url"]

audio_url = upload_audio("enter file path")
print("Uploaded audio URL:", audio_url)


Uploaded audio URL: https://cdn.assemblyai.com/upload/d573db1a-7f4e-4bc9-8f7c-47b107f73e68


In [ ]:
json_data = {
    "audio_url": audio_url,
    "speaker_labels": True  # Enables speaker diarization
}

transcript_response = requests.post(
    "https://api.assemblyai.com/v2/transcript",
    headers=headers,
    json=json_data
)

transcript_id = transcript_response.json()["id"]
print("Transcript ID:", transcript_id)


Transcript ID: d69ee735-32ec-4ebc-98ed-d5f165ec02ce


In [ ]:
import time

def poll_transcript(transcript_id):
    polling_url = f"https://api.assemblyai.com/v2/transcript/{transcript_id}"
    while True:
        res = requests.get(polling_url, headers=headers).json()
        if res['status'] == 'completed':
            return res
        elif res['status'] == 'error':
            raise Exception("Transcription failed:", res)
        print("Waiting for transcript...")
        time.sleep(5)

final_result = poll_transcript(transcript_id)


Waiting for transcript...
Waiting for transcript...
Waiting for transcript...
Waiting for transcript...
Waiting for transcript...


### Transcript with Speakers

In [ ]:
for utterance in final_result['utterances']:
    speaker = utterance['speaker']
    start = round(utterance['start'] / 1000, 2)
    end = round(utterance['end'] / 1000, 2)
    text = utterance['text']
    print(f"Speaker {speaker} | [{start}-{end}] {text}")


Speaker A | [5.12-9.84] Okay so can you guys show us what you.
Speaker B | [10.4-10.88] Yes.
Speaker A | [11.12-11.6] Yeah.
Speaker B | [25.69-25.93] It.
Speaker C | [64.18-64.5] Hello.
Speaker B | [66.82-69.94] How are you doing today? Hello.
Speaker A | [76.83-77.15] Foreign.
Speaker B | [84.83-85.87] How are you today?
Speaker A | [86.67-100.03] Hi, I'm good. So who is going to share like you you said. You guys said that it's done so I just wanted to see.
Speaker B | [101.07-138.71] Yeah, okay. So basically yeah so me and like General we had a like meeting yesterday and we just you know work together and just about the API and today I I also worked and I just finished like signing and like sign up and you know OTB verification stuff. Okay, let me share share you so all of all of you guys here Actor ng ag. Okay so let me. Let me share my screen. So so can you allow me to share my screen?
Speaker D | [139.83-140.39] Yeah.
Speaker B | [142.39-247.94] Okay, thank you. So basically just 